In [1]:
from collections import Counter
from datetime import datetime
import json

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from IntegratedGradients import * 
from keras.models import load_model
import pickle
import numpy as np
from matplotlib import pyplot as plt
# caleb_path = '/Users/CalebKaijiLu/Documents/yelp_dataset/'
caleb_path = '/home/caleb/schoolwork/yelp_dataset/'
selva_path = ''

Using TensorFlow backend.


### Only Run this on Server. If test trained models, use Gender_Prediction_Test.ipynb

In [2]:
def train_model(balanced_texts,balanced_labels,limit):
    tokenizer = Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(balanced_texts)
    sequences = tokenizer.texts_to_sequences(balanced_texts)
    data = pad_sequences(sequences, maxlen=limit)

    model = Sequential()
    model.add(Embedding(20000, 128, input_length=limit))
    model.add(Dropout(0.2))
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(128,dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(data, np.array(balanced_labels), validation_split=0.5, epochs=3,batch_size = 64)
    return tokenizer,model

In [3]:
t1 = datetime.now()
with open(caleb_path + "review.json") as f:
    reviews = f.read().strip().split("\n")
# reviews = reviews[:10000]
reviews = [json.loads(review) for review in reviews]
print(datetime.now() - t1)

0:01:08.541012


In [ ]:
with open(caleb_path + "user_gender.json") as f:
    user_genders = f.read().strip().split("\n")
user_gender = [json.loads(user_gender) for user_gender in user_genders]
user_id_gender = {user['user_id']:user['gender'] for user in user_gender}
print(user_gender[0]['user_id'])
print(user_gender[0]['user_id'] in user_id_pool)

In [ ]:
for review in reviews_gender:

In [ ]:
reviews_gender = [review for review in reviews if review['user_id'] in user_id_gender]
genders = [user_id_gender[review_gender['user_id']] for review_gender in reviews_gender]
bin_gender = [0 if gender =='male' else 1 for gender in genders]
text_gender = [review_gender['text'] for review_gender in reviews_gender]

In [ ]:
balanced_texts_gender = []
balanced_gender = []
limit = 500000  #Change this to grow/shrink the dataset
neg_pos_counts = [0, 0]
for i in range(len(text_gender)):
    polarity = bin_gender[i]
    if neg_pos_counts[polarity]<limit:
        balanced_texts_gender.append(text_gender[i])
        balanced_gender.append(bin_gender[i])
        neg_pos_counts[polarity]+=1
Counter(balanced_gender)

In [ ]:
tokenizer_single, model_single = train_model(balanced_texts_gender,balanced_gender,300)

In [ ]:
tokenizer_user, model_user = train_model(balanced_texts_gender_user,balanced_gender_user,3000)

In [ ]:
with open("tokenizer_gender_single_server.pickle", "wb") as f:
        pickle.dump(tokenizer_single, f)
model_single.save("yelp_gender_single_model_server.hdf5")

In [ ]:
with open("tokenizer_gender_user_server.pickle", "wb") as f:
        pickle.dump(tokenizer_user, f)
model_user.save("yelp_gender_user_model_server.hdf5")